<html><h1>Mapping contributions</h1></html>

This notebook can be used to map contribution receipt data downloaded from the FEC. Simply download a csv from the FEC's website that you are interested in and fill in the four entry prompts in the second cell below. Keep in mind that small dollar donations made through ActBlue or WinRed have been removed from these maps as they skew the data, so this is not a perfectly accurate representation of the contributors, but it's the best we can do without having access to those unitemized contributions.

I ran this notebook using the contribution receipt data from Cal Cunningham's campaign for NC Senate between 2019 and 2020. The preloading the maps made the notebook too large to upload, so I uploaded the html maps to the repository seperately

Last updated: 1/21/2022

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import folium
# from folium.plugins import MousePosition
import json
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup

<html><h4>Entries</h4></html>

In [2]:
csv_path=r'csvs\cunningham-nc-sen-receipts-2019-2020.csv' ##your csv path - 
select_state='NC' #if there is a particular state you want to zoom in on and view results by zip code
raised_or_count='count' #if intent is to view amount 'raised' or 'count' of contributions - both will display on mouseover but only one will be charted on maps
party='Democrat' ## Party of candidate, only relevant for 'Democrat' or 'Republican', no special features for independents or 3rd party candidates. the code should still work with those candidates

<html><h4>Data Wrangling</h4></html>

In [3]:
fundraising_raw=pd.read_csv(csv_path, low_memory=False)
print(fundraising_raw['committee_name'].head(1).iloc[0].title())

fundraising_raw['receipt_date'] = pd.to_datetime(fundraising_raw['contribution_receipt_date']).dt.date
fundraising_raw['receipt_year'] = pd.DatetimeIndex(fundraising_raw['receipt_date']).year
fundraising_raw['receipt_month'] = pd.DatetimeIndex(fundraising_raw['receipt_date']).month
fundraising_raw['receipt_time'] = pd.to_datetime(fundraising_raw['contribution_receipt_date']).dt.time

fundraising_shrink=fundraising_raw[['report_year','report_type','entity_type_desc','contributor_name'
    ,'contributor_first_name','contributor_middle_name','contributor_last_name','contributor_suffix','contributor_street_1'
    ,'contributor_street_2','contributor_city','contributor_state','contributor_zip','contributor_employer'
    ,'contributor_occupation','receipt_date','receipt_year','receipt_month','receipt_time','contribution_receipt_amount','contributor_aggregate_ytd',
    'donor_committee_name','election_type','schedule_type_full']]

# fundraising_shrink1.head(5)

Cal For Nc


In [4]:
# some necessary dictionaries

states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

states_dict = {
        'AK': 'Alaska',
        'AE': 'Armed Forces Europe',
        'AP': 'Armed Forces Pacific',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}


inv_map = {v: k for k, v in states_dict.items()}  ## credit https://stackoverflow.com/questions/483666/reverse-invert-a-dictionary-mapping

overseas = [ 'AE', 'AP', 'GU', 'PR', 'VI', 'MP']


In [5]:
##data used to create geo boundaries and find coordinates

## map of states - doesn't include DC or overseas territories sadly
json_url =     "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json"
response = urlopen(json_url)
state_geo = json.loads(response.read())

##zip code boundaries for state of interest
## if making into a dashboard - improve speed of switching between states by creating a file for each state at the start. cons: takes longer up front, uses more memory, pros: quicker to navigate once loaded.
json_url='https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/'+select_state+"_"+states_dict[select_state].replace(' ','_')+'_zip_codes_geo.min.json' ##Credit Open Data Delaware
response = urlopen(json_url.lower())
zip_boundaries = json.loads(response.read())

##pull coordinates of each state from html table
soup_url='https://www.census.gov/geographies/reference-files/2010/geo/state-area.html' ##Credit US Census Bureau
response=requests.get(soup_url)
soup=BeautifulSoup(response.content,'html')
html_table=soup.find_all('table')
html_table=html_table[0]

coordinates=pd.DataFrame(columns=['state','coords'])

for n in range(len(html_table.tbody.find_all('tr'))-6):
    data=html_table.tbody.find_all('tr')[n+6].find_all('td')
    coordinates.loc[n,'state']=data[0].text.strip()
    if data[-2].text.strip()=='':
        coordinates.loc[n,'coords']=[0,0]
    else:
        coordinates.loc[n,'coords']=[float(data[-2].text.strip()),float(data[-1].text.strip())]

coordinates['state']=coordinates['state'].map(inv_map)
coordinates.dropna(axis=0,inplace=True)

In [6]:
#dataframes for geographies
select_state_full=states_dict[select_state]

geo_df=fundraising_shrink[fundraising_shrink['contributor_state'].notna()]

#ActBlue donations skew data to Massachussetts for democrats, winred to virginia for republicans. need to trim these out for mapping purposes
if party=='Democrat':
    geo_df=geo_df[(geo_df['contributor_name']!='ACTBLUE')] 
elif party=='Republican':
    geo_df=geo_df[(geo_df['contributor_name']!='WINRED')] 

geo_df=geo_df[geo_df['contributor_zip'].notna()]
geo_df=geo_df[~geo_df['contributor_zip'].astype(str).str.contains("[a-zA-Z]").fillna(False)] #drops all zip codes with letters
geo_df=geo_df.astype({'contributor_zip':str})
geo_df['zip+4']=geo_df['zip']=geo_df['contributor_zip'].str[5:] #this also fills in zip codes with < 9 digits, might be worth tinkering with if those +4 zips are relevant
geo_df['zip']=geo_df['contributor_zip'].str[:5]
geo_df.drop('contributor_zip', axis=1)

#breakdown of all zips first, then by state
raised_zip=geo_df[['zip','contribution_receipt_amount']].groupby('zip').sum()
raised_zip.reset_index(inplace=True)
count_zip=geo_df[['zip','contribution_receipt_amount']].groupby('zip').count()
count_zip.reset_index(inplace=True)

state_df=geo_df[geo_df['contributor_state']==select_state]

raised_state_zip=state_df[['zip','contribution_receipt_amount']].groupby('zip').sum()
raised_state_zip.reset_index(inplace=True)
count_state_zip=state_df[['zip','contribution_receipt_amount']].groupby('zip').count()
count_state_zip.reset_index(inplace=True)

raised_state=geo_df[['contributor_state','contribution_receipt_amount']].groupby('contributor_state').sum()
raised_state.reset_index(inplace=True)
# raised_state['state_name']=raised_state['contributor_state'].map(states_dict)
count_state=geo_df[['contributor_state','contribution_receipt_amount']].groupby('contributor_state').count()
count_state.reset_index(inplace=True)
# count_state['state_name']=count_state['contributor_state'].map(states_dict)

raised_usa=raised_state[(raised_state['contributor_state'].isin(states))].reset_index(drop=True)
count_usa=count_state[(count_state['contributor_state'].isin(states))].reset_index(drop=True)

raised_city=geo_df[['contributor_city','contribution_receipt_amount']].groupby('contributor_city').sum()
raised_city.reset_index(inplace=True)
count_city=geo_df[['contributor_city','contribution_receipt_amount']].groupby('contributor_city').count()
count_city.reset_index(inplace=True)


<html><h4>Contribution map of states</h4></html>
This next cell creates a map of the contributions accross the 50 states. I'm hoping to update the GeoJSON to include DC and overseas territories in the future. The map will be centered on the contiguous 48 states, but you can move it around to view Alaska and Hawaii as well.

In [1]:
##add count and raised amount to geojson file - needed to make the mouseover tooltip display this info
for row in state_geo['features']:
    row['properties']['count']=count_usa[count_usa['contributor_state']==row['id']]['contribution_receipt_amount'].iloc[0].astype(str)
    row['properties']['raised']=raised_usa[raised_usa['contributor_state']==row['id']]['contribution_receipt_amount'].iloc[0].astype(str)

usa_map = folium.Map(location=[39, -96], zoom_start=5, tiles="Stamen Watercolor")

map_data=folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=(raised_usa if raised_or_count=='raised' else count_usa),
    columns=["contributor_state", "contribution_receipt_amount"],
    key_on="feature.id",
    fill_color="YlGnBu",
    fill_opacity=0.9,
    line_opacity=0.1,
    legend_name=("Amount Raised ($)" if raised_or_count=='raised' else "Number of Contributions"),
    title='data'
).add_to(usa_map)

folium.features.GeoJsonTooltip(fields=['name','count','raised'],
                               aliases=['State:','Number of Contributions:','Amount Raised ($):']).add_to(map_data.geojson)   #if raised_or_count=='count' else             

usa_map.save('usa_map.html') #- uncomment this to save an html file locally
usa_map

<html><h4>Zip code map of selected state</h4></html>
This next cell will create a map of contributions based on zip code in the state you selected at the top of this notebook. I'm working on improvements that will allow users to switch between states more easily, but this works for now. The zip code boundary files are very large so this loads relatively slowly, but for most states the map will appear in under a minute.

In [2]:
## add count and raised to geojson dictionary (to include these values in mouseover tooltip)
for row in zip_boundaries['features']:
    zip_temp=row['properties']['ZCTA5CE10']
    if zip_temp in count_state_zip.zip.values:
        row['properties']['count']=count_state_zip[count_state_zip['zip']==zip_temp]['contribution_receipt_amount'].iloc[0].astype(str)
        row['properties']['raised']=raised_state_zip[raised_state_zip['zip']==zip_temp]['contribution_receipt_amount'].iloc[0].astype(str)
    else:
        row['properties']['count']='0'
        row['properties']['raised']='0'
        
        
##Map of zip codes in state
state_map = folium.Map(location=(coordinates[coordinates['state']==select_state]['coords'].iloc[0]), zoom_start=7, tiles='cartoDBpositron')

map_data=folium.Choropleth(
    geo_data=zip_boundaries,
    name="choropleth",
    data=(raised_state_zip if raised_or_count=='raised' else count_state_zip),
    columns=["zip", "contribution_receipt_amount"],
    key_on="feature.properties.ZCTA5CE10",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    nan_fill_opacity=0.2,
    line_opacity=0.3,
    legend_name=("Amount Raised ($)" if raised_or_count=='raised' else 'Number of Contributions')
).add_to(state_map)

folium.features.GeoJsonTooltip(fields=['ZCTA5CE10','count','raised'],aliases=['Zip Code:','Number of Contributions:','Amount Raised ($):']).add_to(map_data.geojson)

# state_map.save('state_map.html') #- uncomment this to save an html file locally
state_map